In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
from torchvision import models

/g/g90/tatematsu1/workspace/cardiac_challenge/.venv/lib64/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


---

### model

In [3]:
class model_1(nn.Module):
    def __init__(self, in_channels, in_length, out_channels, out_length, batch_size):
        super().__init__()
        #-----------------------------------
        #var
        self.in_channels = in_channels
        self.in_length = in_length
        self.out_channels = out_channels
        self.out_length = out_length
        self.batch_size = batch_size
        #-----------------------------------
        #Conv1d
        self.in_channels_inner_0 = self.in_channels; self.in_length_inner_0 = self.in_length
        kernel_size=2; stride=1; padding=1; dilation=1; 
        self.out_channels_inner_0 = 20; self.out_length_inner_0 = int((self.in_length_inner_0+2*padding-dilation*(kernel_size-1)-1)/stride+1)
        self.conv1d = nn.Conv1d(in_channels=self.in_channels_inner_0, out_channels=self.out_channels_inner_0, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation)
        #-----------------------------------
        #DNN
        self.in_channels_inner_1 = self.out_channels_inner_0*self.out_length_inner_0
        self.out_channels_inner_1 = self.out_channels*self.out_length

        #-----------------------------------
        #layer0
        self.layer0 = nn.Sequential(
            #-----------------------------------
            nn.Conv1d(in_channels=self.in_channels_inner_0, out_channels=self.out_channels_inner_0, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation),
            #nn.ReLU(),
            nn.LeakyReLU(0.2, inplace=True),
            #nn.BatchNorm1d(num_features=self.in_channels_inner_1),
        )
        #-----------------------------------
        #layer1
        self.layer1 = nn.Sequential(
            #-----------------------------------
            #nn.BatchNorm1d(num_features=self.in_channels_inner_1),
            nn.Linear(self.in_channels_inner_1, self.out_channels_inner_1),
            #nn.ReLU(),
            nn.Sigmoid(),
            #nn.BatchNorm1d(self.params_size+self.features_size),
            #nn.BatchNorm1d(3),
            #nn.LeakyReLU(0.2, inplace=True),
            #nn.Sigmoid()
        )

    def forward(self, x):
        output0 = self.layer0(x).view(-1, self.in_channels_inner_1)
        output1 = self.layer1(output0).view(-1, self.out_channels, self.out_length)
        return output1

In [4]:
"""class model_2(nn.Module):
    def __init__(self, in_channels, in_length, out_channels, out_length, batch_size):
        super().__init__()
        #-----------------------------------
        #var
        self.in_channels = in_channels
        self.in_length = in_length
        self.out_channels = out_channels
        self.out_length = out_length
        self.batch_size = batch_size
        #-----------------------------------
        #DNN
        self.in_channels_inner_1 = self.in_channels*self.in_length
        self.out_channels_inner_1 = 30
        #-----------------------------------
        #DNN
        self.in_channels_inner_2 = self.out_channels_inner_1
        self.out_channels_inner_2 = self.out_channels*self.out_length
        #-----------------------------------
        #layer1
        self.layer1 = nn.Sequential(
            #-----------------------------------
            nn.BatchNorm1d(num_features=self.in_channels_inner_1),
            nn.Linear(self.in_channels_inner_1, self.out_channels_inner_1),
            nn.ReLU(),
            nn.Sigmoid(),
        )
        #-----------------------------------
        #layer2
        self.layer2 = nn.Sequential(
            #-----------------------------------
            nn.BatchNorm1d(num_features=self.in_channels_inner_2),
            nn.Linear(self.in_channels_inner_2, self.out_channels_inner_2),
            nn.ReLU(),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(-1, self.in_channels_inner_1)
        output1 = self.layer1(x)
        output2 = self.layer2(output1)
        output2 = output2.view(-1, self.out_channels, self.out_length)
        return output2"""

'class model_2(nn.Module):\n    def __init__(self, in_channels, in_length, out_channels, out_length, batch_size):\n        super().__init__()\n        #-----------------------------------\n        #var\n        self.in_channels = in_channels\n        self.in_length = in_length\n        self.out_channels = out_channels\n        self.out_length = out_length\n        self.batch_size = batch_size\n        #-----------------------------------\n        #DNN\n        self.in_channels_inner_1 = self.in_channels*self.in_length\n        self.out_channels_inner_1 = 30\n        #-----------------------------------\n        #DNN\n        self.in_channels_inner_2 = self.out_channels_inner_1\n        self.out_channels_inner_2 = self.out_channels*self.out_length\n        #-----------------------------------\n        #layer1\n        self.layer1 = nn.Sequential(\n            #-----------------------------------\n            nn.BatchNorm1d(num_features=self.in_channels_inner_1),\n            nn.Linear(s

### dataset

In [5]:
import os, random
from torch.utils.data import Dataset, DataLoader
from typing import Union

#---------------------------------------------
# custom dataset
#https://discuss.pytorch.org/t/custom-data-loader-for-big-data/129361
class CustomDataset(Dataset):
    def __init__(self, path_dir_X:str, path_dir_Y:str, n_test:Union[int,float], n_val:Union[int,float], batch_size:int): # n_test -> float:ratio of test, int:number of test
        #-----------------
        # batch_size
        self.batch_size = batch_size
        # path_dir_X, path_dir_Y
        self.path_dir_X = path_dir_X
        self.path_dir_Y = path_dir_Y
        # list_file_name_all
        self.list_file_name_all = os.listdir(path_dir_X)
        # n_data_all
        self.n_data_all = len(self.list_file_name_all)
        #check
        if len(os.listdir(path_dir_X)) != len(os.listdir(path_dir_Y)):
            raise ValueError("error!!!")
        if len(set(os.listdir(path_dir_X)) - set(os.listdir(path_dir_Y))) != 0:
            raise ValueError("error!!!")
        #-----------------
        # suffle
        random.shuffle(self.list_file_name_all)
        #-----------------
        # n_test
        if type(n_test)==int:
            self.n_test = n_test
        elif type(n_test)==float:
            self.n_test = int(len(self.list_file_name_all)*n_test)
        else:
            raise ValueError("error!!!")
        # n_val
        if type(n_val)==int:
            self.n_val = n_val
        elif type(n_val)==float:
            self.n_val = int(len(self.list_file_name_all)*n_val)
        else:
            raise ValueError("error!!!")
        #check
        if self.n_data_all <= self.n_test+self.n_val:
            raise ValueError("error!!!")
        #-----------------
        # list_file_name_test / _val / _train
        self.list_file_name_test = self.list_file_name_all[:self.n_test]
        self.list_file_name_val = self.list_file_name_all[self.n_test:self.n_test+self.n_val]
        self.list_file_name_train = self.list_file_name_all[self.n_test+self.n_val:]
        
    def __len__(self):
        return len(self.list_file_name_train)
    
    def __getitem__(self, x):
        #return
        return self.getdata(list_file_name=self.list_file_name_train, index=x)
    
    def getdata(self, list_file_name, index):
        #file_name
        file_name = list_file_name[index]
        #data_X
        path_file_X = "{0}/{1}".format(self.path_dir_X, file_name)
        data_X = np.load(path_file_X, allow_pickle=True)
        data_X = torch.from_numpy(data_X).to(torch.float32).requires_grad_(True)
        #data_Y
        path_file_Y = "{0}/{1}".format(self.path_dir_Y, file_name)
        data_Y = np.load(path_file_Y, allow_pickle=True)
        data_Y = torch.from_numpy(data_Y).to(torch.float32).requires_grad_(True)
        #return
        return data_X, data_Y
    
    def return_n_data_all(self):
        return self.n_data_all
    
    def return_n_test(self):
        return self.n_test
    
    def return_n_val(self):
        return self.n_val
    
    def return_n_train(self):
        return self.n_data_all - self.n_val - self.n_test
    
    def return_batch_size(self):
        return self.batch_size
    
    def return_shape_X(self):
        data_sample = self.getdata(self.list_file_name_all, 0)[0]
        return data_sample.shape
    
    def return_shape_Y(self):
        data_sample = self.getdata(self.list_file_name_all, 0)[1]
        return data_sample.shape
    
    def return_test_data(self):
        #https://www.tutorialspoint.com/how-to-join-tensors-in-pytorch
        data_X_test = torch.stack([self.getdata(self.list_file_name_test, i)[0] for i in range(self.n_test)])
        data_Y_test = torch.stack([self.getdata(self.list_file_name_test, i)[1] for i in range(self.n_test)])
        return data_X_test, data_Y_test
    
    def return_val_data(self):
        #https://www.tutorialspoint.com/how-to-join-tensors-in-pytorch
        data_X_val = torch.stack([self.getdata(self.list_file_name_val, i)[0] for i in range(self.n_val)])
        data_Y_val = torch.stack([self.getdata(self.list_file_name_val, i)[1] for i in range(self.n_val)])
        return data_X_val, data_Y_val


#---------------------------------------------
#var
path_dir_X = "../data_X"
path_dir_Y = "../data_Y_Task3"
n_test = 300
n_val = 300
batch_size = 5000

#---------------------------------------------
#instance
dataset = CustomDataset(path_dir_X=path_dir_X, path_dir_Y=path_dir_Y, n_test=n_test, n_val=n_val, batch_size=batch_size)
dataloder = DataLoader(dataset, batch_size=batch_size, shuffle=True)

### criterion (loss)

In [6]:
#---------------------------------------------
#https://neptune.ai/blog/pytorch-loss-functions
criterion = nn.MSELoss()

### train

In [15]:
#----------------------------
#var (condition)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#----------------------------
#var (train)
num_epochs = 1000
n_print_train_result = 1
val_flag = True
#----------------------------
#var (model)
in_channels = dataset.return_shape_X()[0]
in_length = dataset.return_shape_X()[1]
out_channels = dataset.return_shape_Y()[0]
out_length = dataset.return_shape_Y()[1]
model = model_1(in_channels, in_length, out_channels, out_length, batch_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15,30,45], gamma=0.7)

#----------------------------
#results
history = {"train_loss": [], "val_loss": []}

#----------------------------
#学習
for epoch in range(num_epochs):
  #----------------------------
  # train
  model.train()
  for i, (x, y) in enumerate(dataloder):
    #----------------------------
    #change the type
    x = x.to(device)
    y = y.to(device)
    #----------------------------
    #forward
    output = model(x)
    loss = criterion(output, y)
    #----------------------------
    #backward
    optimizer.zero_grad()
    loss.backward()
    print(x)
    print(x.requires_grad)
    print(x.grad)
    optimizer.step()
    #----------------------------
    #print & result
    if (i+1) % n_print_train_result == 0:
      print(f'Epoch: {epoch+1}, iter: {i+1}, train_loss: {loss: 0.4f}')
    history["train_loss"].append(loss)

  #----------------------------
  # eval
  if val_flag == True:
    model.eval()
    with torch.no_grad():
      #----------------------------
      #forward
      x, y = dataset.return_val_data()
      #----------------------------
      #change the type
      x = x.to(device)
      y = y.to(device)
      #----------------------------
      #forward
      output = model(x)
      loss = criterion(output, y)
      #----------------------------
      #print & result
      history["val_loss"].append(loss)
      print(f'Epoch: {epoch+1}, val_loss: {loss: 0.4f}')
  
  #----------------------------
  # scheduler
  scheduler.step()

tensor([[[-3.0069e-03, -2.1232e-03, -1.4107e-03,  ..., -1.4443e-02,
          -1.4324e-02, -1.4202e-02],
         [-1.4635e-03, -4.6215e-04,  6.6071e-04,  ..., -5.3995e-03,
          -5.3634e-03, -5.3243e-03],
         [ 1.5434e-03,  1.6611e-03,  2.0714e-03,  ...,  9.0440e-03,
           8.9605e-03,  8.8773e-03],
         ...,
         [-1.1774e-03, -8.6344e-04, -5.9488e-04,  ..., -6.9310e-03,
          -6.8726e-03, -6.8131e-03],
         [-4.3417e-04, -1.1648e-04,  2.5163e-04,  ..., -1.5942e-03,
          -1.5836e-03, -1.5720e-03],
         [-1.2808e-04,  2.3355e-04,  6.8275e-04,  ...,  1.1190e-03,
           1.1016e-03,  1.0853e-03]],

        [[ 9.1566e-03,  1.8858e-02,  3.1517e-02,  ...,  2.8919e-03,
           2.6419e-03,  2.4037e-03],
         [ 8.7209e-03,  1.6919e-02,  2.8448e-02,  ...,  3.0386e-03,
           2.7804e-03,  2.5317e-03],
         [-4.3567e-04, -1.9390e-03, -3.0692e-03,  ...,  1.4670e-04,
           1.3858e-04,  1.2793e-04],
         ...,
         [ 1.7271e-03,  3

/var/tmp/tatematsu1/ipykernel_3692965/2359367071.py:44: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(x.grad)


KeyboardInterrupt: 

### result_train

### result_test

---

# Test (CustomDataset)

In [ ]:
import os, random
from torch.utils.data import Dataset, DataLoader
from typing import Union

#---------------------------------------------
# custom dataset
#https://discuss.pytorch.org/t/custom-data-loader-for-big-data/129361
class CustomDataset(Dataset):
    def __init__(self, path_dir_X:str, path_dir_Y:str, n_test:Union[int,float], n_val:Union[int,float], batch_size:int): # n_test -> float:ratio of test, int:number of test
        #-----------------
        # batch_size
        self.batch_size = batch_size
        # path_dir_X, path_dir_Y
        self.path_dir_X = path_dir_X
        self.path_dir_Y = path_dir_Y
        # list_file_name_all
        self.list_file_name_all = os.listdir(path_dir_X)
        # n_data_all
        self.n_data_all = len(self.list_file_name_all)
        #check
        if len(os.listdir(path_dir_X)) != len(os.listdir(path_dir_Y)):
            raise ValueError("error!!!")
        if len(set(os.listdir(path_dir_X)) - set(os.listdir(path_dir_Y))) != 0:
            raise ValueError("error!!!")
        #-----------------
        # suffle
        random.shuffle(self.list_file_name_all)
        #-----------------
        # n_test
        if type(n_test)==int:
            self.n_test = n_test
        elif type(n_test)==float:
            self.n_test = int(len(self.list_file_name_all)*n_test)
        else:
            raise ValueError("error!!!")
        # n_val
        if type(n_val)==int:
            self.n_val = n_val
        elif type(n_val)==float:
            self.n_val = int(len(self.list_file_name_all)*n_val)
        else:
            raise ValueError("error!!!")
        #check
        if self.n_data_all <= self.n_test+self.n_val:
            raise ValueError("error!!!")
        #-----------------
        # list_file_name_test / _val / _train
        self.list_file_name_test = self.list_file_name_all[:self.n_test]
        self.list_file_name_val = self.list_file_name_all[self.n_test:self.n_test+self.n_val]
        self.list_file_name_train = self.list_file_name_all[self.n_test+self.n_val:]
        
    def __len__(self):
        return len(self.list_file_name_train)
    
    def __getitem__(self, x):
        return self.getdata(list_file_name=self.list_file_name_train, index=x)
    
    def getdata(self, list_file_name, index):
        #file_name
        file_name = list_file_name[index]
        #data_X
        path_file_X = "{0}/{1}".format(self.path_dir_X, file_name)
        data_X = np.load(path_file_X, allow_pickle=True)
        data_X = torch.from_numpy(data_X).to(torch.float32)
        #data_Y
        path_file_Y = "{0}/{1}".format(self.path_dir_Y, file_name)
        data_Y = np.load(path_file_Y, allow_pickle=True)
        data_Y = torch.from_numpy(data_Y).to(torch.float32)
        #return
        return data_X, data_Y
    
    def return_n_data_all(self):
        return self.n_data_all
    
    def return_n_test(self):
        return self.n_test
    
    def return_n_val(self):
        return self.n_val
    
    def return_n_train(self):
        return self.n_data_all - self.n_val - self.n_test
    
    def return_batch_size(self):
        return self.batch_size
    
    def return_shape_X(self):
        data_sample = self.getdata(self.list_file_name_all, 0)[0]
        return data_sample.shape
    
    def return_shape_Y(self):
        data_sample = self.getdata(self.list_file_name_all, 0)[1]
        return data_sample.shape
    
    def return_test_data(self):
        #https://www.tutorialspoint.com/how-to-join-tensors-in-pytorch
        data_X_test = torch.stack([self.getdata(self.list_file_name_test, i)[0] for i in range(self.n_test)])
        data_Y_test = torch.stack([self.getdata(self.list_file_name_test, i)[1] for i in range(self.n_test)])
        return data_X_test, data_Y_test
    
    def return_val_data(self):
        #https://www.tutorialspoint.com/how-to-join-tensors-in-pytorch
        data_X_val = torch.stack([self.getdata(self.list_file_name_val, i)[0] for i in range(self.n_val)])
        data_Y_val = torch.stack([self.getdata(self.list_file_name_val, i)[1] for i in range(self.n_val)])
        return data_X_val, data_Y_val


#---------------------------------------------
#var
path_dir_X = "../data_X"
path_dir_Y = "../data_Y_Task3"
n_test = 300
n_val = 300
batch_size = 2

#---------------------------------------------
#instance
dataset = CustomDataset(path_dir_X=path_dir_X, path_dir_Y=path_dir_Y, n_test=n_test, n_val=n_val, batch_size=batch_size)
dataloder = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for i, (data_X, data_Y) in enumerate(dataloder):
   # print(data_X.shape, data_Y.shape)
   print(data_Y)

In [ ]:
dataset.return_n_data_all()

In [ ]:
dataset.return_n_test()

In [ ]:
dataset.return_n_val()

In [ ]:
dataset.return_val_data()[1].shape

In [ ]:
dataset.return_test_data()[1].shape

In [ ]:
x,y = dataset.return_val_data()

---

# Test (model)

In [ ]:
class UNet(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        
        # Encoder
        # In the encoder, convolutional layers with the Conv2d function are used to extract features from the input image. 
        # Each block in the encoder consists of two convolutional layers followed by a max-pooling layer, with the exception of the last block which does not include a max-pooling layer.
        # -------
        # input: 572x572x3
        self.e11 = nn.Conv2d(3, 64, kernel_size=3, padding=1) # output: 570x570x64
        self.e12 = nn.Conv2d(64, 64, kernel_size=3, padding=1) # output: 568x568x64
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 284x284x64

        # input: 284x284x64
        self.e21 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # output: 282x282x128
        self.e22 = nn.Conv2d(128, 128, kernel_size=3, padding=1) # output: 280x280x128
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 140x140x128

        # input: 140x140x128
        self.e31 = nn.Conv2d(128, 256, kernel_size=3, padding=1) # output: 138x138x256
        self.e32 = nn.Conv2d(256, 256, kernel_size=3, padding=1) # output: 136x136x256
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 68x68x256

        # input: 68x68x256
        self.e41 = nn.Conv2d(256, 512, kernel_size=3, padding=1) # output: 66x66x512
        self.e42 = nn.Conv2d(512, 512, kernel_size=3, padding=1) # output: 64x64x512
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 32x32x512

        # input: 32x32x512
        self.e51 = nn.Conv2d(512, 1024, kernel_size=3, padding=1) # output: 30x30x1024
        self.e52 = nn.Conv2d(1024, 1024, kernel_size=3, padding=1) # output: 28x28x1024


        # Decoder
        self.upconv1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.d11 = nn.Conv2d(1024, 512, kernel_size=3, padding=1)
        self.d12 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.upconv2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.d21 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.d22 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.d31 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.d32 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.upconv4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.d41 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.d42 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # Output layer
        self.outconv = nn.Conv2d(64, n_class, kernel_size=1)


    def forward(self, x):
        # Encoder
        xe11 = relu(self.e11(x))
        xe12 = relu(self.e12(xe11))
        xp1 = self.pool1(xe12)

        xe21 = relu(self.e21(xp1))
        xe22 = relu(self.e22(xe21))
        xp2 = self.pool2(xe22)

        xe31 = relu(self.e31(xp2))
        xe32 = relu(self.e32(xe31))
        xp3 = self.pool3(xe32)

        xe41 = relu(self.e41(xp3))
        xe42 = relu(self.e42(xe41))
        xp4 = self.pool4(xe42)

        xe51 = relu(self.e51(xp4))
        xe52 = relu(self.e52(xe51))

        # Decoder
        xu1 = self.upconv1(xe52)
        xu11 = torch.cat([xu1, xe42], dim=1)
        xd11 = relu(self.d11(xu11))
        xd12 = relu(self.d12(xd11))

        xu2 = self.upconv2(xd12)
        xu22 = torch.cat([xu2, xe32], dim=1)
        xd21 = relu(self.d21(xu22))
        xd22 = relu(self.d22(xd21))

        xu3 = self.upconv3(xd22)
        xu33 = torch.cat([xu3, xe22], dim=1)
        xd31 = relu(self.d31(xu33))
        xd32 = relu(self.d32(xd31))

        xu4 = self.upconv4(xd32)
        xu44 = torch.cat([xu4, xe12], dim=1)
        xd41 = relu(self.d41(xu44))
        xd42 = relu(self.d42(xd41))

        # Output layer
        out = self.outconv(xd42)

        return out

In [ ]:
class model_1(nn.Module):
    def __init__(self, in_channels, in_length, out_channels, out_length):
        super().__init__()
        #-----------------------------------
        #Conv1d
        in_channels_inner_0 = in_channels; in_length_inner_0 = in_length
        kernel_size=2; stride=1; padding=1; dilation=1; 
        out_channels_inner_0 = 30; out_length_inner_0 = (in_length_inner_0+2*padding-dilation*(kernel_size-1)-1)/stride+1
        self.conv1d = nn.Conv1d(in_channels=in_channels_inner_0, out_channels=out_channels_inner_0, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation)
        #-----------------------------------
        #DNN
        in_channels_inner_1 = out_channels_inner_0*out_length_inner_0; out_channels_inner_1 = out_channels

        #-----------------------------------
        #layer0
        self.layer0 = nn.Sequential(
            #-----------------------------------
            nn.Conv1d(in_channels=in_channels_inner_0, out_channels=out_channels_inner_0, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation),
            nn.relu(),
            nn.BatchNorm1d(out_channels_inner_0*out_length_inner_0),
            #-----------------------------------
            nn.Linear(in_channels_inner_1, out_channels_inner_1),
            nn.relu(),
            nn.Sigmoid()
            #nn.BatchNorm1d(self.params_size+self.features_size),
            #nn.BatchNorm1d(3),
            #nn.LeakyReLU(0.2, inplace=True),
            #nn.Sigmoid()
        )

    def forward(self, x):
        output0 = self.layer0(x)
        return output0

---

# Test conv1d

In [ ]:
#https://qiita.com/sshuv/items/79d9364b8675fdc080cf
#https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html
#https://cvml-expertguide.net/terms/dl/layers/convolution-layer/


#-----------------------------------
#n_batch=10; n_feature=6; n_time=10; 
n_batch=1000; in_channels=75; in_length=500; 
x = torch.rand(n_batch, in_channels, in_length)

#-----------------------------------
in_channels=in_channels; out_channels=30; kernel_size=3; stride=1; padding=1; dilation=1; 
conv1d = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation)

#-----------------------------------
n_batch=n_batch; out_channels=out_channels; out_length=int((in_length+2*padding-dilation*(kernel_size-1)-1)/stride+1)
y = conv1d(x)
print(n_batch, out_channels, out_length)
print(y.shape)

---

# Test grad

In [11]:
import torch
import torch.nn as nn

input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5, requires_grad=True)
mse_loss = nn.MSELoss()
output = mse_loss(input, target) #mse_loss(input, target), mse_loss(target, input)
output.backward()

print('input: ', input)
print('target: ', target)
print('output: ', output)
print(input.grad)
print(target.grad)

input:  tensor([[-0.5562,  0.6232,  0.4818, -0.6198, -1.1521],
        [-0.4201, -0.5103, -1.0115, -0.8218, -0.4525],
        [-1.6068,  1.5072, -1.0673,  0.4227, -0.5018]], requires_grad=True)
target:  tensor([[-1.3220,  1.7630,  0.7955, -1.6737, -0.9745],
        [-0.8070, -0.5658, -0.2082, -0.9196, -1.0993],
        [-0.0078, -0.2429, -0.4193,  0.8549, -0.9227]], requires_grad=True)
output:  tensor(0.7171, grad_fn=<MseLossBackward0>)
tensor([[ 0.1021, -0.1520, -0.0418,  0.1405, -0.0237],
        [ 0.0516,  0.0074, -0.1071,  0.0130,  0.0862],
        [-0.2132,  0.2333, -0.0864, -0.0576,  0.0561]])
tensor([[-0.1021,  0.1520,  0.0418, -0.1405,  0.0237],
        [-0.0516, -0.0074,  0.1071, -0.0130, -0.0862],
        [ 0.2132, -0.2333,  0.0864,  0.0576, -0.0561]])
